In [1]:
import random
import sys

import pandas as pd

from gcdyn import bdms, gpmap, mutators, responses, utils

sys.path.append("../experiments")

import replay

In [2]:
gp_map = gpmap.ConstantGPMap(1.0)

In [3]:
def make_replay_ish_tree(base_seed):
    tree = bdms.TreeNode()
    tree.sequence = replay.naive_sequence
    tree.x = gp_map(tree.sequence)
    birth_rate = 0.35
    time_to_sampling = 20
    for seed in range(1000):
        try:
            tree.evolve(
                time_to_sampling,
                birth_rate=responses.ConstantResponse(birth_rate),
                mutator=mutators.SequencePhenotypeMutator(replay.mutator, gp_map),
                mutation_rate=replay.mutation_response,
                min_survivors=20,
                seed=base_seed+seed,
            )
            return tree
        except bdms.TreeError:
            print("try failed")
            continue

tree = make_replay_ish_tree(0)
print("tip count: ", len(tree))
tree.sample_survivors(n = 60, seed=0)
tree.prune()

tip count:  509


In [4]:
print(tree)


                     /-1495
                  /-|
               /-|   \-1366
              |  |
            /-|   \-1701
           |  |
           |  |   /-1437
         /-|   \-|
        |  |      \-1226
        |  |
        |  |   /-1193
        |   \-|
        |      \-1542
        |
        |            /-913
        |         /-|
        |        |  |   /-1770
        |        |   \-|
        |      /-|     |   /-1522
        |     |  |      \-|
        |     |  |         \-1799
        |     |  |
      /-|     |   \-1822
     |  |     |
     |  |     |            /-1120
     |  |   /-|         /-|
     |  |  |  |      /-|   \-1197
     |  |  |  |     |  |
     |  |  |  |   /-|   \-948
     |  |  |  |  |  |
     |  |  |  |  |  |   /-1590
     |  |  |  |  |   \-|
     |  |  |  |  |     |   /-1734
     |  |  |   \-|      \-|
     |  |  |     |         \-1697
     |  |  |     |
     |  |  |     |      /-1719
     |  |  |     |   /-|
     |   \-|      \-|   \-1806
     |     |     

In [5]:
tree.total_mutations = 0

for node in tree.iter_descendants(strategy="preorder"):
    node.total_mutations = node.n_mutations + node.up.total_mutations

[leaf.total_mutations for leaf in tree.iter_leaves()]

[5,
 8,
 9,
 12,
 8,
 12,
 10,
 11,
 10,
 12,
 13,
 10,
 11,
 12,
 8,
 8,
 9,
 9,
 7,
 7,
 13,
 4,
 4,
 7,
 6,
 9,
 10,
 8,
 11,
 10,
 9,
 7,
 8,
 12,
 7,
 8,
 12,
 10,
 9,
 14,
 9,
 10,
 9,
 8,
 7,
 8,
 8,
 9,
 9,
 7,
 11,
 10,
 16,
 10,
 13,
 14,
 13,
 11,
 10,
 13]

In [6]:
# Optionally write out tree and sequences.

if False:
    with open("tree.nwk", "w") as fp:
        fp.write(tree.write()+"\n")

    utils.write_leaf_sequences_to_fasta(tree, "seqs.fasta", naive=replay.naive_sequence)